Unisimpls Example
================

Install the `unisimpls` package, then check: 

In [5]:
using unisimpls
print(typeof(predint))

typeof(predint)

1) Set up environment, load data
---------------------------------

Install other dependencies

In [6]:
using CSV
using Statistics
using ScikitLearnBase
using ScikitLearn
using DataFrames
using Parameters

Load data

In [7]:
Xf = CSV.read("../data/Xfearncal.csv", DataFrame, header=0)
yf = CSV.read("../data/Yfearncal.csv", DataFrame, header=0)

Row,Column1
,Float64
1,9.23
2,8.01
3,10.95
4,11.67
5,10.41
6,9.51
7,8.67
8,7.75
9,8.05


2) Unisimpls Regression and Prediction
--------------------------------------

In [10]:
simreg = SIMPLS()
set_params_dict!(simreg,Dict(:n_components => 3,:return_uncertainty => false, :centre => "mean", :scale => "std"))
fit!(simreg,Xf,yf[:,1])
[simreg.fitted_, predict(simreg,Xf)]

SIMPLS
  n_components: Int64 3
  verbose: Bool true
  centre: String "mean"
  scale: String "std"
  copy: Bool true
  all_components: Bool true
  return_uncertainty: Bool false
  store_jacobians: Bool true
  significance: Float64 0.05
  X_Types_Accept: Array{DataType}((7,))
  y_Types_Accept: Array{DataType}((8,))
  X0: Nothing nothing
  y0: Nothing nothing
  Xs_: Nothing nothing
  ys_: Nothing nothing
  x_loadings_: Nothing nothing
  x_scores_: Nothing nothing
  coef_: Nothing nothing
  coef_scaled_: Nothing nothing
  all_coeffs_: Nothing nothing
  intercept_: Nothing nothing
  all_intercepts_: Nothing nothing
  x_ev_: Nothing nothing
  y_ev_: Nothing nothing
  fitted_: Nothing nothing
  fit_pi_: Nothing nothing
  all_fits_: Nothing nothing
  all_fitted: Nothing nothing
  residuals_: Nothing nothing
  all_residuals_: Nothing nothing
  x_Rweights_: Nothing nothing
  x_Rweights_unscaled_: Nothing nothing
  x_Vloadings_: Nothing nothing
  x_Vloadings_unscaled_: Nothing nothing
  colret_: 

2-element Vector{Vector{Float64}}:
 [9.321972444930502, 8.098889100949108, 10.889836919419192, 11.25242089365102, 10.096721463459076, 9.220761920905776, 9.08648565384545, 7.771552479960285, 7.73945637128984, 11.475140660270768  …  11.790370653287603, 8.178771525871412, 12.522704706555704, 8.632856887304982, 9.938606637877125, 11.50796329469765, 9.886234972992469, 10.774063412277101, 10.924706835080304, 11.506185530373081]
 [9.321972444930502, 8.098889100949108, 10.889836919419192, 11.25242089365102, 10.096721463459076, 9.220761920905776, 9.08648565384545, 7.771552479960285, 7.73945637128984, 11.475140660270768  …  11.790370653287603, 8.178771525871412, 12.522704706555704, 8.632856887304982, 9.938606637877125, 11.50796329469765, 9.886234972992469, 10.774063412277101, 10.924706835080304, 11.506185530373081]

3) Cross-Validation
-------------------

The `unisimpls` package is written to be consistent with `ScikitLearn.jl`, such that widely used functions from ScikitLearn can be applied to it. This includes the cross-validation setup.

In [20]:
import ScikitLearn.GridSearch:GridSearchCV
simgrid = SIMPLS(return_uncertainty=false, scale="std")
gridsearch = GridSearchCV(simgrid, Dict(:n_components => collect(1:6)))
fit!(gridsearch,Matrix(Xf),yf[:,1])
gridsearch.best_params_
predict(gridsearch.best_estimator_,Xf)

Fitting a 1 component SIMPLS model
Calculating JacobiansFitting a 1 component SIMPLS model
Calculating JacobiansFitting a 1 component SIMPLS model
Calculating JacobiansFitting a 2 component SIMPLS model
Calculating JacobiansFitting a 2 component SIMPLS model
Calculating JacobiansFitting a 2 component SIMPLS model
Calculating JacobiansFitting a 3 component SIMPLS model
Calculating JacobiansFitting a 3 component SIMPLS model
Calculating JacobiansFitting a 3 component SIMPLS model
Calculating JacobiansFitting a 4 component SIMPLS model
Calculating JacobiansFitting a 4 component SIMPLS model
Calculating JacobiansFitting a 4 component SIMPLS model
Calculating JacobiansFitting a 5 component SIMPLS model
Calculating JacobiansFitting a 5 component SIMPLS model
Calculating JacobiansFitting a 5 component SIMPLS model
Calculating JacobiansFitting a 6 component SIMPLS model
Calculating JacobiansFitting a 6 component SIMPLS model
Calculating JacobiansFitting a 6 component SIMPLS model
Calculating J

24-element Vector{Float64}:
  9.36384272280911
  8.131988079074588
 10.897786513436316
 11.153353298626431
 10.191936435085566
  9.268099326529924
  9.092584126442901
  7.758678268124676
  7.827455607242349
 11.481006837129424
 10.02236306848065
  7.952483306355351
 10.424267812006299
 10.169543655611019
 11.798622089657442
  8.091248079267512
 12.570415993195216
  8.528072254651974
  9.933569573416957
 11.507012938652636
  9.878673960072653
 10.741387713753667
 10.935500510023289
 11.470107830354085

4) Uncertainty Estimation
-------------------------
Uncertainty estimation is accomplished through error propagation using the fastest algorithm available to calculate Jacobian matrices. 
When `return_uncertainty` and `store_jacobians` are both set to `true`, the resulting Jacobians will be available in the object: 

In [21]:
simreg = SIMPLS()
set_params_dict!(simreg,Dict(:n_components => 3,:return_uncertainty => true, :centre => "mean", :scale => "std"))
fit!(simreg,Xf,yf[:,1])
simreg.dbdy_

SIMPLS
  n_components: Int64 3
  verbose: Bool true
  centre: String "mean"
  scale: String "std"
  copy: Bool true
  all_components: Bool true
  return_uncertainty: Bool true
  store_jacobians: Bool true
  significance: Float64 0.05
  X_Types_Accept: Array{DataType}((7,))
  y_Types_Accept: Array{DataType}((8,))
  X0: Nothing nothing
  y0: Nothing nothing
  Xs_: Nothing nothing
  ys_: Nothing nothing
  x_loadings_: Nothing nothing
  x_scores_: Nothing nothing
  coef_: Nothing nothing
  coef_scaled_: Nothing nothing
  all_coeffs_: Nothing nothing
  intercept_: Nothing nothing
  all_intercepts_: Nothing nothing
  x_ev_: Nothing nothing
  y_ev_: Nothing nothing
  fitted_: Nothing nothing
  fit_pi_: Nothing nothing
  all_fits_: Nothing nothing
  all_fitted: Nothing nothing
  residuals_: Nothing nothing
  all_residuals_: Nothing nothing
  x_Rweights_: Nothing nothing
  x_Rweights_unscaled_: Nothing nothing
  x_Vloadings_: Nothing nothing
  x_Vloadings_unscaled_: Nothing nothing
  colret_: N

6×24 Matrix{Float64}:
  0.0387999    0.0214383  -0.0569864   …  -0.167584   -0.0647936  -0.137242
  0.0276706   -0.143341    0.163319        0.0567056   0.163202    0.14515
  0.00260346  -0.0473614   0.172226        0.0598432   0.139039    0.114746
  0.0141768    0.14049    -0.232862       -0.206384   -0.213669   -0.209063
  0.124537     0.152895   -0.00726857      0.1764      0.06243    -0.0424717
 -0.228024    -0.155051   -0.0791472   …   0.119238   -0.100484    0.140122

It is easy to verify that this Jacobian is identical to the ones obtained through numerical differentiation, yet calculated much faster: 

In [26]:
using FiniteDifferences
Xs = autoscale(Xf,"mean","std").X_as_
ys = autoscale(yf,"mean","std").X_as_
fdy(y) = unisimpls._fit_simpls(3,6,24,Matrix(Xs),y,[],true,false)[7]
J = jacobian(backward_fdm(5,1),fdy,ys[:,1])[1]

6×24 Matrix{Float64}:
  0.0387999    0.0214383  -0.0569864   …  -0.167584   -0.0647936  -0.137242
  0.0276706   -0.143341    0.163319        0.0567056   0.163202    0.14515
  0.00260346  -0.0473614   0.172226        0.0598432   0.139039    0.114746
  0.0141768    0.14049    -0.232862       -0.206384   -0.213669   -0.209063
  0.124537     0.152895   -0.00726857      0.1764      0.06243    -0.0424717
 -0.228024    -0.155051   -0.0791472   …   0.119238   -0.100484    0.140122

The Jacobian with respect to X: 

In [27]:
simreg.dbdX_

6×144 Matrix{Float64}:
 -0.0745805    -0.875218    1.44079   …  -0.308043   -0.300589   -0.267144
 -0.000469315   0.059091   -0.13697      -0.122004   -0.0679141  -0.0858584
  0.0111489     0.0813813  -0.171287     -0.118578   -0.0621579  -0.0748911
 -0.0374821     0.484338   -0.925503     -0.444493   -0.543179   -0.498568
  0.0954279     0.0407961   0.148205      0.0356389  -0.0590981  -0.128461
 -0.0124172     0.180885   -0.387574  …   0.983349    1.01672     1.05974

Prediction interval for training data: 

In [28]:
simreg.fit_pi_

24×2 Matrix{Float64}:
  7.84978  10.7942
  6.06963  10.1281
  8.77309  13.0066
  8.83856  13.6663
  8.25095  11.9425
  8.14964  10.2919
  7.37297  10.8
  5.85664   9.68646
  6.85998   8.61893
 10.6961   12.2541
  8.24881  11.7424
  5.7938   10.1434
  8.57917  12.1899
  7.96362  12.4875
  7.59119  15.9896
  6.90243   9.45511
  5.81347  19.2319
  7.43034   9.83538
  9.08231  10.7949
 10.7193   12.2967
  8.06068  11.7118
  9.1198   12.4283
  9.80692  12.0425
 10.512    12.5003

This can equivalently be obtained by using the function `predint`, which will calculate prediction intervals for training, test or new data points and allows to specify a significance lavel different from the one in the parent `SIMPLS` object. 

In [30]:
predint(simreg,Xf)

24×2 Matrix{Float64}:
  7.84978  10.7942
  6.06963  10.1281
  8.77309  13.0066
  8.83856  13.6663
  8.25095  11.9425
  8.14964  10.2919
  7.37297  10.8
  5.85664   9.68646
  6.85998   8.61893
 10.6961   12.2541
  8.24881  11.7424
  5.7938   10.1434
  8.57917  12.1899
  7.96362  12.4875
  7.59119  15.9896
  6.90243   9.45511
  5.81347  19.2319
  7.43034   9.83538
  9.08231  10.7949
 10.7193   12.2967
  8.06068  11.7118
  9.1198   12.4283
  9.80692  12.0425
 10.512    12.5003

In [33]:
predint(simreg,Xf,.1)

24×2 Matrix{Float64}:
  8.08647  10.5575
  6.39588   9.8019
  9.11341  12.6663
  9.22665  13.2782
  8.5477   11.6457
  8.32185  10.1197
  7.64846  10.5245
  6.16451   9.37859
  7.00138   8.47754
 10.8214   12.1289
  8.52965  11.4616
  6.14345   9.79376
  8.86942  11.8996
  8.32728  12.1238
  8.26631  15.3144
  7.10763   9.24991
  6.89214  18.1533
  7.62367   9.64204
  9.21998  10.6572
 10.8461   12.1699
  8.35418  11.4183
  9.38576  12.1624
  9.98663  11.8628
 10.6719   12.3405